<h1><center>Performance Assessment Task 1 - Linear Regression Modeling</center></h1>
<h3><center> by Bader Ale <center><h3>

# ▶ Research Question
Our research questions is as follows: __What patient factors contributed to the highest total charge billed to the patient during their hospital stay.__.

In [ ]:
# Importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.interactiveshell import InteractiveShell # Importing so we can run multiple lines in one cell
InteractiveShell.ast_node_interactivity = "all" # Code so multiple lines in one cell can be ran simultaenously 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing original dataset
df = pd.read_csv('F:/GitHub Repos/WGU_MSDA/D208_Predictive Modeling/medical_clean.csv')

# ▶ Data Cleaning  

In [ ]:
# Showing first 5 records
df.head()

In [ ]:
# Creating new dataframe with only those variables of interest
df1 = df[['Area','Age', 'Income','Marital', 'Gender', 'VitD_levels', 'Doc_visits', 'Initial_admin','Complication_risk', 'Overweight', 'Arthritis', 'Diabetes', 'Hyperlipidemia', 'Asthma','Services','Initial_days', 'TotalCharge']]
df1.head()

## Detection and Treatment of Nulls

In [ ]:
# Getting number of rows and columns
df1.shape

In [ ]:
# Checking for null values
df1.isnull().sum()

Here we can see there are no Nulls in our new dataframe

## Detection and Treatment of Duplicated Values

In [ ]:
# Checking for duplicates
df1.duplicated().value_counts()

The output shows 10000 records as being False, therefore we do not have any duplicated values.

## Detection and Treatment of Outliers

In [ ]:
# Checking datatypes for all variables in new dataframe
df1.dtypes

We will first focus on the continuous variables and analyze, if any, the outliers. We will normalize our data for ease of visual interpretation

In [ ]:
# Performing normalization on the continuous variables for the new dataframe
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df1[['Age', 'Income', 'VitD_levels','Doc_visits','Initial_days','TotalCharge']]) # Selecting only those numerical variables in our new dataframe
scaled = scaler.fit_transform(df1[['Age', 'Income', 'VitD_levels','Doc_visits','Initial_days','TotalCharge']])
df1_norm = pd.DataFrame(scaled, columns = ['Age', 'Income', 'VitD_levels','Doc_visits','Initial_days','TotalCharge']) # Creating a new dataframe for visualization

In [ ]:
# Plotting boxplots
sns.boxplot(df1_norm)
plt.title('Numeric Variables')
plt.ylabel('Count(Normalized)')
plt.xlabel('Explanatory Variables');

In [ ]:
# Importing SciPy library
import scipy.stats as stats

# Creating a new column for the Income and VitaminD_level z-scores
df1['Income_z_Scores'] = stats.zscore(df1['Income'])
df1['VitD_level_z_Scores'] = stats.zscore(df1['VitD_levels'])

In [ ]:
df1.head()

In [ ]:
# Creating a new dataframe with extracted Income and VitD_levels outliers
df1_no_outliers = df1[(df1['Income_z_Scores'] > -3) & (df1['Income_z_Scores'] < 3) & (df1['VitD_level_z_Scores'] > -3) & (df1['VitD_level_z_Scores'] < 3)]

In [ ]:
df1_no_outliers.head()

In [ ]:
# Dropping the Income_z_Scores and Vit_D_levels columns since
# we won't need it anymore
df1_no_outliers.drop(['Income_z_Scores', 'VitD_level_z_Scores'], axis=1, inplace=True)

df1_no_outliers.head()

# ▶ Exploratory Data Analysis  

In [ ]:
# Summary Statistics
df1_no_outliers.describe()

In [ ]:
df1_no_outliers.dtypes

## Univariate Analysis  
### Numeric Variables

In [ ]:
# Univariate analysis visualizations for numerical variables
sns.displot(df1_no_outliers['Age']);
sns.displot(df1_no_outliers['Income']);
sns.displot(df1_no_outliers['VitD_levels']);
sns.displot(df1_no_outliers['Doc_visits']);
sns.displot(df1_no_outliers['Initial_days']);
sns.displot(df1_no_outliers['TotalCharge']);

### Categorical Variables

In [ ]:
sns.histplot(df1_no_outliers['Area']);

In [ ]:
sns.histplot(df1_no_outliers['Marital']);

In [ ]:
sns.histplot(df1_no_outliers['Gender']);

In [ ]:
sns.histplot(df1_no_outliers['Initial_admin']);

In [ ]:
sns.histplot(df1_no_outliers['Services']);

In [ ]:
sns.histplot(df1_no_outliers['Complication_risk']);

In [ ]:
sns.histplot(df1_no_outliers['Overweight']);

In [ ]:
sns.histplot(df1_no_outliers['Arthritis']);

In [ ]:
sns.histplot(df1_no_outliers['Diabetes']);

In [ ]:
sns.histplot(df1_no_outliers['Hyperlipidemia']);

In [ ]:
sns.histplot(df1_no_outliers['Asthma']);

## Bivariate Analysis  
### Numeric Variables

In [ ]:
# Bivariate analysis visualizations for Age vs Total_Charge
sns.jointplot(data=df1_no_outliers, x='Age', y='TotalCharge')
plt.title('Age vs. Total Charge')
fig = plt.gcf()
fig.set_size_inches(16, 10);

In [ ]:
# Income vs Total Charge
sns.jointplot(data=df1_no_outliers, x='Income', y='TotalCharge')
plt.title('Income vs. Total Charge')
plt.xlabel('Annual Income of Patient (USD)')
fig = plt.gcf()
fig.set_size_inches(16, 10);

In [ ]:
# VitaminD Levels vs Total Charge
sns.jointplot(data=df1_no_outliers, x='VitD_levels', y='TotalCharge')
plt.title('Vitamin D Levels vs. Total Charge')
plt.xlabel('Vitamin D Levels (ng/mL)')
fig = plt.gcf()
fig.set_size_inches(16, 10);

In [ ]:
# Doc-Visits vs Total Charge
sns.jointplot(data=df1_no_outliers, x='Doc_visits', y='TotalCharge')
plt.title('Doctor Visits vs. Total Charge')
plt.xlabel('Number of Doctor Visits')
fig = plt.gcf()
fig.set_size_inches(16, 10);

In [ ]:
# Doc-Visits vs Total Charge
sns.jointplot(data=df1_no_outliers, x='Initial_days', y='TotalCharge')
plt.title('Length of Stay vs. Total Charge')
plt.xlabel('Length of Stay (days)')
fig = plt.gcf()
fig.set_size_inches(16, 10);

### Categorical Variables

In [ ]:
df1_no_outliers.head()

In [ ]:
sns.catplot(data=df1_no_outliers, x="Area", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Area vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Marital", y="TotalCharge", kind='bar', errorbar=None, height=4, aspect=2)
plt.title('Marital vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Gender", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Gender vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Initial_admin", y="TotalCharge", kind='bar', errorbar=None,height=4, aspect=2)
plt.title('Method of Admission vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Complication_risk", y="TotalCharge", kind='bar', errorbar=None,height=4, aspect=2)
plt.title('Complication Risk vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Overweight", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Overweight vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Arthritis", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Arthritis vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Diabetes", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Diabetes vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Hyperlipidemia", y="TotalCharge", kind='bar', errorbar=None)
plt.title('High Blood Pressure vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Asthma", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Asthma vs. Total Charge');

In [ ]:
sns.catplot(data=df1_no_outliers, x="Services", y="TotalCharge", kind='bar', errorbar=None)
plt.title('Medical Services vs. Total Charge');

# ▶ Data Wrangling  
In this section, we will rexpress the categorical variables 

In [ ]:
# Printing datatypes 
df1_no_outliers.dtypes

In [ ]:
# Using One-Hot encoding for nominal variables
df2 = pd.get_dummies(data=df1_no_outliers, columns=['Area','Marital','Gender', 'Initial_admin','Overweight','Arthritis', 'Services', 'Overweight', 'Diabetes', 'Hyperlipidemia', 'Asthma'], drop_first=True )

In [ ]:
# Using Ordinal Encoding for ordinal variables
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
complication_encoded = enc.fit_transform(df2[['Complication_risk']])
df2['Complication_risk'] = complication_encoded

df2.head(10)


In [ ]:
# Exporting dataframe to CSV
df2.to_csv('model_data.csv',index=True)

# ▶ Initial Multiple Linear Regression Model  
We will now construct our initial model

In [ ]:
import statsmodels.api as sm
import seaborn as sns

# Creating feature variables, where X = independent variables and Y=dependent variables
X_data = df2.drop('TotalCharge', axis=1)
Y_data = df2['TotalCharge']

print('The shape of the features is:',X_data.shape)
X_data.head()
print('The shape of the labels:',Y_data.shape)
Y_data.head()

In [ ]:
# Adding constant to X_data
X_data = sm.add_constant(X_data)
#np.asarray(X_data)

# Fitting regression model 
model = sm.OLS(Y_data,np.asarray(X_data))
#model.fit()

#print(model.summary())

In [ ]:
model_prediction = model.predict(X_data)
model_prediction.shape
model_prediction.head()

In [ ]:
# Actual vs. Predicted Values for Initial Model
sns.histplot(Y_data, label='Actual Values')
sns.histplot(model_prediction, label="Predicted Values", alpha=0.5)
plt.title('Actual vs Predicted Values for Initial Model')
plt.legend();

In [ ]:
model.params

Initial Regression Model Equation:

TotalCharge = 2358.675577 + 0.060576(Age) +  0.000005(Income) + 0.220942(VitD_levels) + 1.222980(Doc_visits) - 200.577435(Complication_risk) + 81.929811(Initial_days)  
              + 3.975710(Area_Suburban) + 4.727445(Area_Urban) + 4.514919(Marital_Married) - 3.355894(Marital_Never Married) - 4.737488(Marital_Separated) 
              - 5.767059(Marital_Widowed) - 0.520453(Gender_Male) + 13.478394(Gender_Nonbinary) + 511.564613(Initial_admin_Emergency Admission) 
              - 1.771107(Initial_admin_Observation Admission) + 2.337127(Overweight_Yes) + 72.877156(Arthritis_Yes) + 8.360456(Services_CT Scan) 
              - 2.740121(Services_Intravenous) - 1.166686(Services_MRI) + 2.337127(Overweight_Yes) + 71.513515(Diabetes_Yes) + 93.168755(Hyperlipidemia_Yes) 
              + 2.945512(Astha_Yes)

### Feature Selection

In [ ]:
# Visualizing correlation heatmap in seaborn
sns.heatmap(df2.corr(),annot=True, fmt=".1f", cmap='Blues')
plt.title('Correlation Heat Map')
fig = plt.gcf()
fig.set_size_inches(18,16);

In [ ]:
# Performing Recursive Feature Elimination to reduce number of explanatory variable

# Importing necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE


x_rfe = df2.drop('TotalCharge', axis=1)
y_rfe = df2[['TotalCharge']]

selector = RFE(estimator=RandomForestRegressor(), step=1, n_features_to_select=3)
selector = selector.fit(x_rfe,y_rfe,)

In [ ]:
# Printing what features were selected by the RFE process
selector.get_feature_names_out()

# ▶ Reduced Multiple Linear Regression Model

In [ ]:
# Running reduced regression model with features named in RFE above
X_data2 = df2[['Complication_risk', 'Initial_days', 'Initial_admin_Emergency Admission']]
Y_data2 = df2[['TotalCharge']]

print('The shape of the features is:',X_data2.shape)
X_data2.head()
print('The shape of the labels:',Y_data2.shape)
Y_data2.head()

In [ ]:
# Adding constant to X_data2
X_data2 = sm.add_constant(X_data2)

# Fitting regression model 
model2 = sm.OLS(Y_data2,X_data2).fit()

# Creating prediction for reduced model 
model2_prediction = model2.predict(X_data2)

In [ ]:
# Actual vs. Predicted Values for Reduced Model
sns.histplot(Y_data2, label='Actual Values')
sns.histplot(model2_prediction, label="Predicted Values", alpha=0.5)
plt.title('Actual vs Predicted Values for Reduced Model')
plt.xlabel('TotalCharge')
plt.legend();

In [ ]:
print(model2.summary())

In [ ]:
model2.params

Reduced Regression Model Equation:

TotalCharge = 2451.189539 - 199.260061(Complication_risk) + 81.945552(Initial_days) 
              + 513.432540(Initial_admin_Emergency Admission)

# ▶ Residual Plots (Task I)

In [ ]:
# Initial Model vs Reduced Model Predictions
sns.histplot(model_prediction, label='Initial Model')
sns.histplot(model2_prediction, label="Reduced Model", alpha = 0.5)
plt.title('Initial Model Predictions vs Reduced Model Predictions')
plt.legend();

### Initial Regression Model

In [ ]:
# Q-Q Plot of Initial Regression Model
sm.qqplot(model.resid, line='s')
plt.title('QQ Plot : Initial Regression Model')
plt.show();

In [ ]:
# Residual Mean Square Error  
model.bse

### Reduced Regression Model

In [ ]:
# Q-Q Plot
sm.qqplot(model2.resid, line='s')
plt.title('QQ Plot : Reduced Regression Model')
plt.show();

In [ ]:
# Residual Mean Square Error  
model2.bse

# ▶ Initial Logistic Regression Model  
Research Question: **What patient factors cause high cholesterol?**

In [ ]:
df2.head()

In [ ]:
sns.barplot(data=df2, x="Complication_risk", y="Hyperlipidemia_Yes", errorbar=None,);

In [ ]:
sns.barplot(data=df2, x="Area_Suburban", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Area_Urban", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Marital_Married", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Marital_Never Married", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Marital_Separated", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Marital_Widowed", y="Hyperlipidemia_Yes", errorbar=None);


In [ ]:
sns.barplot(data=df2, x="Gender_Male", y="Hyperlipidemia_Yes", errorbar=None);

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Creating X and Y vairbales for the logistic regression
X_data_log = df2.drop('Hyperlipidemia_Yes', axis=1)
Y_data_log = df2['Hyperlipidemia_Yes']

# Using SKlearn to split th data to test and train
X_log_train, X_log_test, Y_log_train, Y_log_test = train_test_split(X_data_log, Y_data_log, test_size=0.3, random_state=100)

# Scaling data
scaler = StandardScaler()
X_log_train_scaled = scaler.fit_transform(X_log_train)
X_log_test_scaled = scaler.transform(X_log_test)

#Running Logistic Regression
log_reg = LogisticRegression(random_state=0, fit_intercept=True, solver="liblinear", C=1.0).fit(X_log_train_scaled, Y_log_train)

In [ ]:
log_reg.classes_
log_reg.intercept_
log_reg.coef_

In [ ]:
# Scoring Model using training data
log_reg.score(X_log_train_scaled, Y_log_train)

In [ ]:
# Scoring Model using test data
log_reg.score(X_log_test_scaled, Y_log_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Y_log_test, log_reg.predict(X_log_test))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues');

# ▶ Reduced Logistic Regression Model 

In [ ]:
# Creating X and Y variables for the logistic regression
X_data2_log = df2[['Complication_risk', 'Initial_days', 'Initial_admin_Emergency Admission']]
Y_data2_log = df2['Hyperlipidemia_Yes']

# Using SKlearn to split th data to test and train
X2_log_train, X2_log_test, Y2_log_train, Y2_log_test = train_test_split(X_data2_log, Y_data2_log, test_size=0.3, random_state=100)

# Scaling data
scaler2 = StandardScaler()
X2_log_train_scaled = scaler.fit_transform(X2_log_train)
X2_log_test_scaled = scaler.transform(X2_log_test)

# Running Model
log_reg_reduced = LogisticRegression(random_state=0, fit_intercept=True, class_weight = None).fit(X2_log_train_scaled, Y2_log_train)

In [ ]:
log_reg_reduced.classes_
log_reg_reduced.intercept_
log_reg_reduced.coef_

In [ ]:
# Scoring Model using training data
log_reg_reduced.score(X2_log_train_scaled, Y2_log_train)

In [ ]:
# Scoring Model using testing data
log_reg_reduced.score(X2_log_test_scaled, Y2_log_test)

In [ ]:
cf_matrix = confusion_matrix(Y2_log_test, log_reg_reduced.predict(X2_log_test))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues');

Reduced Logistic Model Equation:

ln(y/(1-y)) = -0.66695895 + 0.02214502(Complication Risk) - 0.03500518(Initial_Days) + 0.00724142(Initial_admin_Emergency Admission)